# APITable Langchain Toolkit

This is [Langchain](https://python.langchain.com/) toolkits allows agents to interact with APITable Fusion API, performing actions such as get spaces, get datasheets, get records, create fields and create datasheet. The tool wraps the [apitable.py](https://github.com/apitable/apitable-sdks/tree/develop/apitable.py) library. 

And it is in beta, please don't use it in production.

To use this tool, you must first set as environment variables: APITABLE_API_TOKEN
```python
os.environ["OPENAI_API_KEY"] = "xxx"
os.environ["APITABLE_API_TOKEN"] = "xxx"
```



In [1]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent

from langchain.chat_models import ChatOpenAI

from langchain.callbacks import get_openai_callback

from apitable_toolkit.tool.prompt import PREFIX, FORMAT_INSTRUCTIONS, SUFFIX
from apitable_toolkit.toolkit import APITableToolkit
from apitable_toolkit.utilities.apitable import APITableAPIWrapper


In [4]:
llm = ChatOpenAI(temperature=0)
apitable = APITableAPIWrapper()
toolkit = APITableToolkit.from_apitable_api_wrapper(apitable)

In [3]:
def run(prompt):
    tools = toolkit.get_tools(prompt)
    print([t.name for t in tools])

    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        agent_kwargs={
            "prefix": PREFIX,
            "format_instructions": FORMAT_INSTRUCTIONS,
            "suffix": SUFFIX,
        },
    )

    with get_openai_callback() as cb:
        try:
            agent.run(prompt)
        except Exception as e:
            print(e)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Successful Requests: {cb.successful_requests}")
        print(f"Total Cost (USD): ${cb.total_cost}")

In [5]:
run("What spaces do I have")

['Create Datasheets', 'Create Fields', 'Get Spaces', 'Get Nodes']


> Entering new AgentExecutor chain...
Thought: I need to use the Get Spaces tool to fetch all the spaces I have access to.
Action: Get Spaces
Action Input: No input required
Observation: Found 4 spaces:
[{'id': 'spctqtTZpssYw', 'name': "xukecheng's Space", 'isAdmin': True}, {'id': 'spcS0eZxZ8mSA', 'name': 'APITable Ltd.', 'isAdmin': None}, {'id': 'spcGSVizcwRYF', 'name': 'Gmail Space', 'isAdmin': None}, {'id': 'spcV5VkzU71Lf', 'name': 'test03-bug验证升级', 'isAdmin': None}]
Thought:I have found all the spaces I have access to, so I can now answer the original question.
Final Answer: I have access to 4 spaces: "xukecheng's Space", "APITable Ltd.", "Gmail Space", and "test03-bug验证升级".

> Finished chain.
Total Tokens: 3446
Prompt Tokens: 3355
Completion Tokens: 91
Successful Requests: 2
Total Cost (USD): $0.0068920000000000006


In [6]:
run("Tell me the latest value of APITable MAU in xukecheng's space")

['Get Fields', 'Get Records', 'Get Spaces', 'Get Nodes']


> Entering new AgentExecutor chain...
Thought: We need to find the latest value of APITable MAU in xukecheng's space. We need to use the Get Fields tool to get the field key for APITable MAU first.
Action: Get Fields
Action Input: datasheet_id for xukecheng's space
Observation: Found a error 'Expecting value: line 1 column 1 (char 0)', you should only output Action Input in JSON format
Thought:We need to input the datasheet_id for xukecheng's space in JSON format to use the Get Fields tool.
Action: Get Fields
Action Input: {"datasheet_id": "dstxxxxxxxxxxxxxx"}
Observation: Found a error 'the specified datasheet does not exist', please try another tool to get right datasheet_id.
Thought:We need to use the Get Nodes tool to find the correct datasheet_id for APITable MAU in xukecheng's space.
Action: Get Nodes
Action Input: space_id for xukecheng's space
Observation: Found a error 'Expecting value: line 1 column 1 (char 0)', you s

In [9]:
run("Create a datasheet for Project management in xukecheng's space")

['Create Datasheets', 'Create Fields', 'Get Spaces', 'Get Nodes']


> Entering new AgentExecutor chain...
Thought: I need to use the Create Datasheets tool to create a datasheet for Project management in xukecheng's space. I need to input the space id and the name of the datasheet.
Action: Create Datasheets
Action Input: space_id = xukecheng's space id, name = "Project management"
Observation: Found a error 'Expecting value: line 1 column 1 (char 0)', you should only output Action Input in JSON format
Thought:I need to input the space id and the name of the datasheet in JSON format.
Action: Create Datasheets
Action Input: {"space_id": "xukecheng's space id", "name": "Project management"}
Observation: Found a error 'Action input need effective space_id', please try another tool to get right space_id.
Thought:I need to use the Get Spaces tool to get the correct space_id for xukecheng's space.
Action: Get Spaces
Action Input: None
Observation: Found 4 spaces:
[{'id': 'spctqtTZpssYw', 'nam

In [12]:
run("Create a datasheet for Project management in xukecheng's space, need to include the following fields: name, description, start_date, end_date, status, owner, priority, progress, comments")

['Create Datasheets', 'Create Fields', 'Get Spaces', 'Get Nodes']


> Entering new AgentExecutor chain...
Thought: We need to use the Create Datasheets tool to create a datasheet in xukecheng's space. We also need to determine the field types and properties for each field we want to include in the datasheet.

Action: Create Datasheets
Action Input: 
- space id: xukecheng's space id
- name: "Project management"
- field_data: 
  - name: "name"
    type: "SingleText"
    property: 
      defaultValue: ""
  - name: "description"
    type: "Text"
  - name: "start_date"
    type: "DateTime"
    property:
      dateFormat: "YYYY-MM-DD"
      includeTime: false
  - name: "end_date"
    type: "DateTime"
    property:
      dateFormat: "YYYY-MM-DD"
      includeTime: false
  - name: "status"
    type: "SingleSelect"
    property:
      options: []
  - name: "owner"
    type: "Member"
    property:
      isMulti: false
      shouldSendMsg: false
  - name: "priority"
    type: "SingleSelect"
    p